In [1]:
# STEP 0: importing required packages
import polars as pl
import json
import os
import polars.selectors as cs
import xlsxwriter
import zipfile
import io
from pathlib import Path

STEP 1: Defining the path to the required files

In [51]:
# 1.1 json file containing skillcorner competition editions information
ce_path = Path('skillcorner-competition-editions.json')

In [52]:
# 1.2 Directory containing json files with skillcorner position data
json_PosDir_path = Path('skillcorner-20250214.zip')

In [53]:
# 1.3 csv containing statsbomb competition id and info (country & division)
sb_path = "./statsbomb-competition-levels.csv"

In [54]:
# 1.4 csv containing comparison of statsbomb and skillcorner id's / information
sb_sc_path = "./mapping-competition-seasons.csv"

STEP 2: Converting the competitions editions json to a usable DataFrame

In [55]:

with ce_path.open('r') as ce_file:
    ce_data = json.load(ce_file)['results']
    ce_df = pl.json_normalize(ce_data)

STEP 3: Combining the json files with the position data into a usable dataframe

In [56]:
# initializing an empty list to store json files
json_list = []

In [57]:
# Looping through the json files and storing them into one list
with zipfile.ZipFile(json_PosDir_path, "r") as zip:
    for file_name in zip.namelist():
        if file_name.endswith(".json"):
            with zip.open(file_name) as f:
                json_bytes = f.read()
                json_buffer = io.BytesIO(json_bytes)
                json_df = pl.read_json(json_buffer)
                json_list.append(json_df)

STEP 4: Combining the data

In [137]:
# concatenate json files and join with ce_df (keep left on column name)
df = pl.concat(
        json_list
    ).join(
        ce_df, 
        left_on = "competition_edition_id",
        right_on = "id",
        how = "left"
    )


In [138]:
# --> mag nog weg, hier zit de limitatie niet
pl.Config.set_tbl_rows(150)
# 'name',
# 'competition.id',
# 'competition.area',
# 'competition.name',
# 'competition.gender',
# 'competition.age_group',
df['competition_edition_id'].unique()

competition_edition_id
i64
249
353
357
360
374
375
376
377
379


STEP 5: Cobining statsbomb and skillcorner data with the physical data

In [139]:
# sb_sc_path: sc_competition_season_id bevat null values

In [140]:
# terug verwijderen
sb = pl.read_csv(sb_sc_path)

In [141]:
# terug verwijderen
sb.head()

sb_competition_id,sb_competition_name,sb_season_id,sb_season_name,sc_competition_id,sc_competition_name,sc_season_id,sc_season_name,sc_competition_season_id,sc_competition_season_name
i64,str,i64,str,i64,str,i64,str,i64,str
2,"""Premier League""",317,"""2024/2025""",1,"""Premier League""",95,"""2024/2025""",895,"""ENG - Premier League - 2024/20…"
2,"""Premier League""",281,"""2023/2024""",1,"""Premier League""",28,"""2023/2024""",543,"""ENG - Premier League - 2023/20…"
2,"""Premier League""",235,"""2022/2023""",1,"""Premier League""",21,"""2022/2023""",387,"""ENG - Premier League - 2022/20…"
2,"""Premier League""",108,"""2021/2022""",1,"""Premier League""",8,"""2021/2022""",287,"""ENG - Premier League - 2021/20…"
2,"""Premier League""",90,"""2020/2021""",1,"""Premier League""",7,"""2020/2021""",147,"""ENG - Premier League - 2020/20…"


In [142]:
# terug verwijderen
sb['sc_competition_name'].unique()

sc_competition_name
str
"""Primera Division"""
"""Ligat Al"""
"""Nemzeti Bajnokság"""
"""Primeira Liga"""
"""Pro League"""
"""LaLiga 3"""
"""Championship"""
"""Liga Dimayor"""
"""Major League Soccer"""


In [143]:
sb = pl.read_csv(sb_path)
sb_sc = pl.read_csv(sb_sc_path)

In [144]:
#debugging
df = sb_sc.join(sb, #join statsbomb information with the sb - sc comparison
        left_on="sb_competition_id",
        right_on="competition_id",
        how = "left"
).select(
    [
        "sc_competition_season_id", 
        "competition_region_name",
        "competition_division_level"
    ]
)

df.head()

sc_competition_season_id,competition_region_name,competition_division_level
i64,str,str
895,"""England""","""1"""
543,"""England""","""1"""
387,"""England""","""1"""
287,"""England""","""1"""
147,"""England""","""1"""


In [155]:
missing_sb_ids = []

for id in sb_sc['sb_competition_id']:
    if id not in sb['competition_id']:
        missing_sb_ids.append(id)

print(list(set(missing_sb_ids)))

[16, 274, 1426, 280, 283, 286, 292, 295, 1449, 300, 1337, 1338, 1339, 1340, 1341, 1595, 218, 353, 226, 231, 1259, 1264, 119, 121, 125]


In [161]:
missing_sb_ids_dflist = []

for id in missing_sb_ids:
    missing_sb_ids_dflist.append(sb_sc.filter(pl.col("sb_competition_id")==id)
)


In [162]:
missing_sb_ids_df = pl.concat(missing_sb_ids_dflist)

In [164]:
missing_sb_ids_df

with xlsxwriter.Workbook("missing_statsbomb_data.xlsx") as wb:
    missing_sb_ids_df.write_excel(
        workbook=wb, 
        autofit = True,
        float_precision = 1,
        freeze_panes = (1,0),
        header_format = {"bold": True}
    )


In [149]:
# kijken welke competition uit the statsbomb scillcorner mapping file niet in de statsbomb file zit

In [134]:
# 5.1 Linking competition info (country & division) to skillcorner competition edition
df = sb_sc.join(sb, #join statsbomb information with the sb - sc comparison
        left_on="sb_competition_id",
        right_on="competition_id",
        how = "left"
).select(
    [
        "sc_competition_season_id", 
        "competition_region_name",
        "competition_division_level"
    ]
).with_columns(
    pl.concat_str(
        [
            pl.col("competition_region_name"),
            pl.col("competition_division_level").cast(str),
        ],
        separator=" ",
    ).alias("competition_region_division")
).drop(
    [
        "competition_region_name", 
        "competition_division_level"
    ]
).join(
    df, 
    left_on="sc_competition_season_id",
    right_on="competition_edition_id",
    how = "right"
)

In [135]:
df['competition_region_division'].unique()

competition_region_division
str
"""Portugal 2"""
"""Germany 3"""
"""France 1"""
"""Germany 1"""
"""Spain 3"""
"""Uruguay 1"""
"""Belgium 1"""
"""Romania 1"""
"""Portugal 1"""


In [86]:
df.head()

sc_competition_season_id,competition_region_division,player_id,Count Sprint TIP 1,Sprinting Distance OTIP 2,player_name,Distance TIP 2,Count Medium Deceleration TIP,short_name,Count High Deceleration TIP 1,position,Count Sprint 1,HSR Distance TIP 2,Minutes,Count High Deceleration OTIP 2,Count Medium Deceleration TIP 2,Count Medium Deceleration TIP 1,Count Sprint OTIP 2,Count Medium Deceleration 2,Count High Deceleration,Count HSR TIP 1,Count High Deceleration 1,Count Medium Deceleration OTIP 2,team,Running Distance 1,Count HSR 2,Count HSR OTIP 1,Count HSR OTIP 2,group,match_id,Count High Acceleration 1,Minutes TIP,date,HSR Distance OTIP 1,player_birthdate,Distance 1,Distance 2,…,Count High Deceleration TIP 2,Count High Deceleration OTIP 1,season_id,Running Distance OTIP 1,PSV-99,Sprinting Distance,Minutes 2,Minutes OTIP 1,HSR Distance TIP,team_name,quality_check,Count High Acceleration 2,Count HSR TIP 2,competition_id,Running Distance OTIP,Running Distance OTIP 2,Distance TIP 1,HSR Distance 2,Count Medium Acceleration OTIP,Sprinting Distance TIP,Minutes OTIP,season_name,Count HSR 1,Count Medium Acceleration TIP 2,Count High Deceleration OTIP,Minutes TIP 2,Count Medium Acceleration TIP,name,competition.id,competition.area,competition.name,competition.gender,competition.age_group,season.id,season.start_year,season.end_year,season.name
i64,str,i64,i64,i64,str,i64,i64,str,i64,str,i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,str,i64,i64,f64,str,i64,str,i64,i64,…,i64,i64,i64,i64,f64,i64,f64,f64,i64,str,bool,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,str,i64,i64,i64,f64,i64,str,i64,str,str,str,str,i64,i64,i64,str
895,"""England 1""",13078,2,37,"""Mason Mount""",549,16,"""M. Mount""",4,"""AM""",10,68,61.38,2,0,16,1,8,18,10,15,2,"""Manchester United""",933,7,17,2,"""Midfield""",1650385,4,22.05,"""2024-08-16""",233,"""1999-01-10""",5763,1723,…,0,2,95,482,30.3,321,15.18,10.88,194,"""Manchester United""",true,0,3,1,638,156,2503,104,21,81,14.36,"""2024/2025""",30,3,4,3.67,19,"""ENG - Premier League - 2024/20…",1,"""ENG""","""Premier League""","""male""","""adult""",95,2024,2025,"""2024/2025"""
895,"""England 1""",24579,4,41,"""Conor Chaplin""",461,11,"""C. Chaplin""",2,"""AM""",7,26,65.51,2,0,11,2,16,15,12,10,6,"""Ipswich Town""",1112,9,18,7,"""Midfield""",1650961,2,14.25,"""2024-08-17""",239,"""1997-02-16""",6043,2259,…,0,3,95,646,29.2,316,19.23,15.18,214,"""Ipswich Town""",true,0,0,1,990,344,1922,149,33,115,21.75,"""2024/2025""",31,3,5,2.55,13,"""ENG - Premier League - 2024/20…",1,"""ENG""","""Premier League""","""male""","""adult""",95,2024,2025,"""2024/2025"""
895,"""England 1""",13589,2,73,"""Dominik Szoboszlai""",2381,25,"""D. Szoboszlai""",1,"""AM""",8,217,99.88,3,14,11,3,42,19,18,9,10,"""Liverpool Football Club""",1080,26,13,14,"""Midfield""",1650961,4,32.58,"""2024-08-17""",198,"""2000-10-25""",5884,5633,…,2,2,95,520,32.0,510,53.6,11.7,448,"""Liverpool Football Club""",true,2,11,1,973,453,2279,460,31,190,21.85,"""2024/2025""",33,20,5,17.4,35,"""ENG - Premier League - 2024/20…",1,"""ENG""","""Premier League""","""male""","""adult""",95,2024,2025,"""2024/2025"""
895,"""England 1""",24703,2,21,"""João Pedro Junqueira de Jesus""",1650,24,"""João Pedro""",2,"""AM""",3,145,85.78,1,11,13,2,36,22,14,12,6,"""Brighton and Hove Albion""",1205,22,19,10,"""Midfield""",1651702,5,25.74,"""2024-08-17""",203,"""2001-09-26""",6135,4186,…,3,3,95,572,29.3,234,36.6,13.83,328,"""Brighton and Hove Albion""",true,5,11,1,824,252,2018,285,32,160,21.35,"""2024/2025""",36,15,4,11.87,33,"""ENG - Premier League - 2024/20…",1,"""ENG""","""Premier League""","""male""","""adult""",95,2024,2025,"""2024/2025"""
895,"""England 1""",12166,5,20,"""Morgan Gibbs-White""",2066,30,"""M. Gibbs-White""",2,"""AM""",9,156,107.9,3,19,11,1,50,18,13,9,15,"""Nottingham Forest""",973,33,18,15,"""Midfield""",1651704,4,28.77,"""2024-08-17""",189,"""2000-01-27""",5952,5605,…,2,2,95,454,27.5,169,51.0,13.58,330,"""Nottingham Forest""",true,4,18,1,888,434,2009,399,40

STEP 6: Performance metrics have to be normalized towards a 90 minute match based on the minutes played

In [19]:
# 6.1: Defining column names that do not have to be normalized (non numeric columns)
list2skip = df.select(~cs.numeric()).columns

In [20]:
# Hardcoding the numerical columns that do not have to be normalized
hardcoded = [
    'PSV-99',
    'competition_edition_id',
    'competition_id',
    'competition_id_right',
    'match_id',
    'player_id',
    'season_id_right',
    'season_id',
    'season_end_year',
    'season_start_year',
    'team_id'
]

In [21]:
list2skip = list2skip + hardcoded

In [23]:
# 6.2 hardcoding the time indicators
# Needed to correctly normalize each metric
time_indicators = {
    "OTIP 2": "Minutes OTIP 2",
    "OTIP 1": "Minutes OTIP 1",
    "TIP 2": "Minutes TIP 2",
    "TIP 1": "Minutes TIP 1",
    "OTIP": "Minutes OTIP",
    "TIP": "Minutes TIP",
    "2": "Minutes 2",
    "1": "Minutes 1"
}

In [24]:
# iterating over the columns to match them to the correct time / minute column
# normalizing the metrics for the time played
for coln in df.columns:
    if coln in list2skip:
        continue

    for pattern, indicator in time_indicators.items():
        if pattern in coln:
            divisor = indicator
            break
        else:
            divisor = "Minutes"

    df = df.with_columns(
        ((pl.col(coln) / pl.col(divisor))*90).alias(f"P90 {coln}")
    )

In [25]:
# sort the dataframe again for logical column order
df = df.select(sorted(df.columns))

STEP 7: Creating Excell files showing benchmarked data for the needed metrics

In [26]:
# 7.1 group the symmetrical positions by hardcoding
# symmetrical positions have to be shown in the same excell sheet 
mapping = {
    "RM": "RM|LM",
    "LM": "RM|LM",
    "RCB": "RCB|LCB",
    "LCB": "RCB|LCB",
    "RWB": "RWB|LWB",
    "LWB": "RWB|LWB",
    "RF": "RF|LF",
    "LF": "RF|LF",
    "RW": "RW|LW",
    "LW": "RW|LW",
    "DM": "DM",
    "AM": "AM",
    "CB": "CB",
    "CF": "CF"
}

In [27]:
df = df.with_columns(
    position_grouped = pl.col("position").replace_strict(mapping)
)

In [28]:
# 7.2 Hardcoding column names of the metrics wanted in the excells
# competition_region_division and position are fixed columns and should not be changed
df_subset = df.select([
    'competition_region_division', 
    'position_grouped',
    'P90 Distance',
    'P90 Running Distance',
    'P90 HSR Distance', 
    'P90 Sprinting Distance', 
    'PSV-99'
])

In [29]:
# The performance metrics are variable columns and can be modified
# depending on the wanted analysis
metrics = df_subset.drop([
    'competition_region_division', 
    'position_grouped'
]).columns

In [30]:
# 7.3 Hardcoding positions in the same order as the wanted excell sheet order
positions = ['CB', 'RCB|LCB', 'RWB|LWB', 'DM','RM|LM', 'AM', 'RW|LW', 'CF', 'RF|LF']

In [38]:
def position_filter(dataframe, position):
    """Filters the dataframe on a specified position"""
    pos_df = dataframe.filter(
                pl.col("position_grouped") == position
            ).filter(
                pl.col("competition_region_division").is_not_null()
            )
    
    return pos_df

In [39]:
def sample_size_calculator(dataframe):
    """Calculate the sample size per position per competition"""
    ss_df = (
        dataframe.group_by(
            "competition_region_division"        
        ).agg([
            pl.col("competition_region_division").len().alias("Sample Size")
        ])
    )
    
    return ss_df

In [31]:

# 7.4 Create the 1st excell type
# store the statistical measures to loop through 
stats = {
    "Mean": pl.col(metrics).mean(),
    "Median": pl.col(metrics).median(),
    "Std": pl.col(metrics).std(),
    "Q25": pl.col(metrics).quantile(0.25),
    "Q75": pl.col(metrics).quantile(0.75),
}

In [44]:
with xlsxwriter.Workbook("match_benchmarks_stat_grouped.xlsx") as wb:

    for position in positions:
        result_list = []
        
        # Filter for the specific position
        df_pos = position_filter(df_subset, position)

        # determine the sample size per competition
        ss = sample_size_calculator(df_pos)

        for stat_name, stat_expr in stats.items():
            result = (
                df_pos
                .group_by("competition_region_division")
                .agg([stat_expr])
                .with_columns(pl.lit(stat_name).alias("Statistical Measure"))
            )
            result_list.append(result)

        # Combine all results with columns in prefered order 
        pl.concat(
                result_list
            ).join(
                ss, on="competition_region_division"
            ).select([
                    "competition_region_division",
                    "Statistical Measure",
                    "Sample Size"
                ]
                + metrics
            ).rename({
                "competition_region_division":"Competition"
            }).write_excel(
                workbook=wb, 
                worksheet = position,
                autofit = True,
                float_precision = 1,
                freeze_panes = (1,0),
                header_format = {"bold": True}
            )

In [45]:
# 7.5 Create the 2nd excell type
with xlsxwriter.Workbook("match_benchmarks_stat_sep.xlsx") as wb:

    for position in positions:    
        result_list = []

        # Filter for the specific position
        df_pos = position_filter(df_subset, position)

        # determine the sample size per competition
        ss = sample_size_calculator(df_pos)
        result_list.append(ss)

        # Filter for this specific position
        for metric in metrics:
            result = (
                df_pos
                .filter(pl.col("position_grouped") == position)
                .group_by("competition_region_division")
                .agg(
                    [pl.col(metric).mean().alias(f"{metric} Mean")] +
                    [pl.col(metric).median().alias(f"{metric} Median")] + 
                    [pl.col(metric).std().alias(f"{metric}  Std")] + 
                    [pl.col(metric).quantile(0.25).alias(f"{metric} Q25")]+
                    [pl.col(metric).quantile(0.75).alias(f"{metric} Q75")]
                )
            )   

            result_list.append(result)

        # Combine all results in a dataframe to write to a position sheet of the excel
        pl.concat(
                result_list, how='align'
            ).rename({
                "competition_region_division":"Competition"
            }).write_excel(
                workbook=wb, 
                worksheet = position,
                autofit = True,
                float_precision = 1,
                freeze_panes = (1,0),
                header_format = {"bold": True}
            )

In [42]:
# STEP 9: Write the dataframe to a parquet file that will be used for visualisation
# entire dataframe written to the parquet file 
parquet4visual_path = "./parquet4visual.parquet"

In [43]:

df.write_parquet(parquet4visual_path)